In [1]:
%matplotlib inline

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
from numpy import genfromtxt
import pywt
from datetime import time, datetime, timedelta
import random
import itertools
import io
import json
import cvxopt

from classification import *
from kernel import *

In [2]:
Xtr = genfromtxt('../data/Xtr.csv', delimiter=',')
Ytr = genfromtxt('../data/Ytr.csv', delimiter=',')
Xte = genfromtxt('../data/Xte.csv', delimiter=',')

Xtr = np.delete(Xtr, 3072, axis=1)
Xte = np.delete(Xte, 3072, axis=1)
Ytr = Ytr[1:,1]
N = len(Ytr)

## Test new SVM

In [ ]:
#features_functions = [fourier_modulus_1D_kernel, fourier_modulus_2D_kernel, fourier_phase_2D_kernel, wavelet_transform]
#lambdas = [10, 1, 0.1, 0.01]
#features_functions = [fourier_modulus_2D_kernel]
features_functions = [scattering_transform]
lambdas = [0.01]
submission = True

results_cross_val = {}

for feature_function in features_functions:
    Xtr_t = feature_function(Xtr)
    for _lambda in lambdas:
        results_cross_val[(feature_function, _lambda)] = []
        for i in range(1):
            
            if submission:
                Xte_t = feature_function(Xte)
                features = Xtr_t.T
                labels = Ytr
                X = Xte_t.T
            else:
                mask_test = random.sample(range(0, 5000), 1000)
                mask_train =  [i for i in range(5000) if i not in mask_test]

                Xtr_train = Xtr_t[mask_train, :].T
                Xtr_test = Xtr_t[mask_test, :].T
                Ytr_train = Ytr[mask_train]
                Ytr_test = Ytr[mask_test]
                
                features = Xtr_train
                labels = Ytr_train
                X = Xtr_test

            t1 = datetime.now()
            alphas, bias = one_versus_all_SVM(features, labels, _lambda=_lambda)
            print 'model fitted'
            prediction = predict_SVM(alphas, bias, features, X)
            t2 = datetime.now()
            print t2-t1
            
            if submission == False:
                well_classified = 0
                for i in range(len(prediction)):
                    if prediction[i] == Ytr_test[i]:
                         well_classified+=1
                print 'lambda = ', _lambda, ', good classification rate = ', float(well_classified)/len(Ytr_test)
            else:
                results_cross_val[(feature_function, _lambda)].append(well_classified)

In [ ]:
DF = pd.DataFrame(data=pd.Series(prediction), columns=['Prediction'])
DF.index += 1
DF.to_csv('../data/'+'scattering_transform_gabor.csv', index=True, index_label='Id', sep=',')

## Toy data for tests

In [ ]:
n_samples = 150
mean_1 = [0, -3]
cov = [[3, 0], [0, 3]]
X_1 = np.random.multivariate_normal(mean_1, cov, n_samples).T
mean_2 = [3, 3]
X_2 = np.random.multivariate_normal(mean_2, cov, n_samples).T
mean_3 = [-3, 3]
X_3 = np.random.multivariate_normal(mean_3, cov, n_samples).T
X = np.concatenate((X_1, X_2, X_3), axis = 1)

y = np.concatenate((np.zeros((1,n_samples)), np.ones((1, n_samples)), 2*np.ones((1,n_samples))), axis=1)
y = y[0,:]
Xtr_t = X.T
Ytr = y

mask_test = range(0,450, 5)
mask_train = [i for i in range(450) if i not in mask_test]

Xtr_train = Xtr_t[mask_train, :].T
Xtr_test = Xtr_t[mask_test, :].T
Ytr_train = Ytr[mask_train]
Ytr_test = Ytr[mask_test]